## Contents

- [Updates](#Updates)
- [Wrangle](#Wrangle)
- [Target](#Target)
- [Recode](#Recode)
- [Comingle learning columns](#Comingle-learning-columns)
- [Aggregate](#Aggregate)
- [Set Lists](#Set-Lists)
- [Results](#Results)


## Updates
[*Back to top*](#Contents)

In [1]:
# Python updates

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import itertools as it
import re

In [2]:
# Local updates

import sys
sys.path.append('../')
import wrangle
import explore

In [3]:
 # Jupyter notebooks only #
def ifull(obj):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(obj)

## Wrangle
[*Back to top*](#Contents)

In [4]:
df, dictdf = wrangle.wrangle_data()
print(df.shape)

(726, 87)


In [5]:
print(dictdf.shape)
# ifull(dictdf.describe().T)

(90, 11)


## Target
[*Back to top*](#Contents)

In [6]:
df = explore.add_target_to_df(df)

In [7]:
print(df.target.value_counts())

1.0    372
0.5    202
0.0    152
Name: target, dtype: int64


In [8]:
print(df.targetb.value_counts())

1    372
0    354
Name: targetb, dtype: int64


## Recode
[*Back to top*](#Contents)

In [9]:
dfrecode2 = explore.make_recode_df(df=df, data_dictionary=dictdf, outputs = 2)

## Comingle learning columns
[*Back to top*](#Contents)

In [10]:
learn_by_cols = {
    'learning_talks': '12za',
    'learning_read': '12zb',
    'learning_meetup': '12zc',
    'learning_workshop': '12zd',
    'learning_conference': '12ze',
    'likely_watch_video': '12za',
    'likely_internet': '12zb',
    'likely_book': '12zb',
    'likely_online_group': '12zf',
    'likely_colleague': '12zg',
    'likely_meetup': '12zc',
    'likely_conference': '12ze',
    'likely_workshop': '12zd',
}
learn_by_groups = {
    '12za': 'learn_by_video', 
    '12zb': 'learn_by_reading', 
    '12zc': 'learn_by_meetup', 
    '12zd': 'learn_by_workshop', 
    '12ze': 'learn_by_conference', 
    '12zf': 'learn_by_online_group', 
    '12zg': 'learn_by_colleague', 
}

In [11]:
use_cols = [col for col in learn_by_cols]
comingle_df = explore.comingle_groups_max(
    df=dfrecode2[use_cols].copy(), 
    cols_to_group=learn_by_cols, 
    group_to_col=learn_by_groups)
print(comingle_df.shape)

(726, 7)


In [12]:
dfrecode2 = dfrecode2.join(comingle_df)

## Aggregate
[*Back to top*](#Contents)

In [13]:
agg_df = dfrecode2.copy()
agg_df.targetb = agg_df.targetb.astype('object')
display(agg_df.head())
agg_df = agg_df.groupby(['targetb']).agg(['mean']).T
display(agg_df.head())
agg_df = agg_df.reset_index()
display(agg_df.head())
agg_df.index_name='idx'
display(agg_df.head())
agg_df = agg_df.rename(columns={
    'level_0': 'field', 'level_1': 'stat'
})
display(agg_df.head())
agg_df = agg_df.melt(id_vars=['field', 'stat'], var_name='targetb', value_name='calc_val')
display(agg_df.head())
agg_df = agg_df.pivot_table(values='calc_val', index=['field','targetb'], columns='stat')
display(agg_df.head())

# ifull(agg_df)

,job_title,job_taxo,job_id,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,...,persona_id,target,targetb,learn_by_video,learn_by_reading,learn_by_meetup,learn_by_workshop,learn_by_conference,learn_by_online_group,learn_by_colleague
resp_id,,,,,,,,,,,,,,,,,,,,,
284,Academic,4,1.0,1,1,0,1,1,1,1,...,2,1.0,1,0,1,1,1,1,1,1
288,Academic,4,1.0,0,1,0,1,1,1,1,...,2,1.0,1,1,1,1,1,1,1,1
294,Academic,4,1.0,1,1,0,1,0,1,1,...,2,1.0,1,1,1,1,1,1,1,1
281,Academic,4,1.0,1,1,0,1,1,1,0,...,2,1.0,1,1,1,1,1,1,1,1
280,Academic,4,1.0,1,1,0,0,0,1,1,...,2,1.0,1,1,1,0,0,1,0,1


,targetb,0,1
job_conduct_res,mean,0.802260,0.876344
job_analyze_res,mean,0.833333,0.911290
job_buy_res_report,mean,0.067797,0.067204
job_manage_res_proj,mean,0.751412,0.798387
job_observe_res,mean,0.536723,0.607527


targetb,level_0,level_1,0,1
0,job_conduct_res,mean,0.802260,0.876344
1,job_analyze_res,mean,0.833333,0.911290
2,job_buy_res_report,mean,0.067797,0.067204
3,job_manage_res_proj,mean,0.751412,0.798387
4,job_observe_res,mean,0.536723,0.607527


targetb,level_0,level_1,0,1
0,job_conduct_res,mean,0.802260,0.876344
1,job_analyze_res,mean,0.833333,0.911290
2,job_buy_res_report,mean,0.067797,0.067204
3,job_manage_res_proj,mean,0.751412,0.798387
4,job_observe_res,mean,0.536723,0.607527


targetb,field,stat,0,1
0,job_conduct_res,mean,0.802260,0.876344
1,job_analyze_res,mean,0.833333,0.911290
2,job_buy_res_report,mean,0.067797,0.067204
3,job_manage_res_proj,mean,0.751412,0.798387
4,job_observe_res,mean,0.536723,0.607527


,field,stat,targetb,calc_val
0,job_conduct_res,mean,0,0.802260
1,job_analyze_res,mean,0,0.833333
2,job_buy_res_report,mean,0,0.067797
3,job_manage_res_proj,mean,0,0.751412
4,job_observe_res,mean,0,0.536723


stat                       mean
field         targetb          
company_large 0        0.477401
              1        0.456989
company_med   0        0.225989
              1        0.206989
company_small 0        0.293785

## Set Lists
[*Back to top*](#Contents)

In [14]:
qs_sessions = [
    'session_keynote',
    'session_long_talk',
    'session_short_talk',
    'session_workshop',
    'session_network',
    'session_social_event',
    'session_qa',
    'sesson_topic_tables',
]

In [15]:
qs_factors = [
    'factor_speaker',
    'factor_diverse_speak',
    'factor_topics',
    'factor_format_sessions',
    'factor_size',
    'factor_network',
    'factor_variety_attend',
    'factor_code',
    'factor_location',
    'factor_ability_to_pay',
]

In [16]:
qs_learning = [
    'learn_by_video',
    'learn_by_reading',
    'learn_by_meetup',
    'learn_by_workshop',
#     'learn_by_conference',
    'learn_by_online_group',
    'learn_by_colleague'
]

## Results
[*Back to top*](#Contents)

In [17]:
idx = pd.IndexSlice

In [18]:
targets = {
    0: 'Not likely to attend',
    1: 'Likely to attend'
}
for l in [qs_sessions, qs_factors, qs_learning]: 
    for b in targets:
        print(f'{b}: {targets[b]}')
        display(agg_df.loc[idx[l, [b]], ['mean']].sort_values(by='mean', ascending=False).head(5))

0: Not likely to attend


,stat,mean
field,targetb,
session_workshop,0,0.731638
session_keynote,0,0.692090
session_long_talk,0,0.669492
sesson_topic_tables,0,0.564972
session_short_talk,0,0.497175


1: Likely to attend


,stat,mean
field,targetb,
session_workshop,1,0.763441
session_long_talk,1,0.741935
session_keynote,1,0.693548
sesson_topic_tables,1,0.580645
session_short_talk,1,0.564516


0: Not likely to attend


...

,stat,mean
field,targetb,
factor_topics,1,0.978495
factor_ability_to_pay,1,0.838710
factor_diverse_speak,1,0.774194
factor_location,1,0.706989
factor_format_sessions,1,0.672043


0: Not likely to attend


,stat,mean
field,targetb,
learn_by_reading,0,0.994350
learn_by_video,0,0.937853
learn_by_colleague,0,0.903955
learn_by_meetup,0,0.703390
learn_by_workshop,0,0.635593


1: Likely to attend


,stat,mean
field,targetb,
learn_by_reading,1,1.000000
learn_by_colleague,1,0.938172
learn_by_video,1,0.935484
learn_by_workshop,1,0.924731
learn_by_meetup,1,0.865591
